In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from keras import models, layers, optimizers
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import punkt
stop_words = stopwords.words('english')
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

data = '../input/fake-set/train_1.tsv'
data = pd.read_table(data)
print(data.shape)

data.columns = ['Json','Label','News','Variable_1','Variable_2','Variable_3','Variable_4','Variable_5'
    ,'Variable_6','Variable_7','Variable_8','Variable_9','Variable_10','Variable_11']

data = data.drop(['Variable_1','Variable_2','Variable_3','Variable_4','Variable_5'
    ,'Variable_6','Variable_7','Variable_8','Variable_9','Variable_10','Variable_11'],axis=1)

data = data.drop(['Json'],axis=1)

print(data.head())

X, y = data.loc[:, data.columns == 'News'], data.Label
Xtrain,Xtest,y_train,y_test = train_test_split(data['News'],y,test_size=0.2,shuffle=False)

In [ ]:
!pip install bert-for-tf2 

!pip install sentencepiece

In [ ]:
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert

In [ ]:
data = pd.read_csv('../input/fake-news-detection/train.csv')
print('Training data shape: ', data.shape)
data.head()

In [ ]:
from sklearn.model_selection import train_test_split
X,y = data.loc[:, data.columns!='label'],data.label
Xtrain,Xtest,Ytrain,Ytest = train_test_split(X['text'],y,test_size=0.2)

## Exploratory Data Analysis

In [ ]:
#Missing values in training set
train.isnull().sum()

In [ ]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
            
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [ ]:
Ytrain[0]

In [ ]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=True)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [ ]:
tokenizer.tokenize("don't be so judgmental")

In [ ]:
Xtrain = Xtrain.apply(lambda x: x.lower())
Xtest = Xtest.apply(lambda x: x.lower())

In [ ]:
Xtrain = Xtrain.fillna(',')
Xtest = Xtest.fillna(',')

In [ ]:
all_tokens = []
all_masks = []
all_segments = []
embedd = []
all_clf = []

max_len = 512
for text in Xtest:
    text = tokenizer.tokenize(text)

    text = text[:max_len-2]
    input_sequence = ["[CLS]"] + text + ["[SEP]"]
    pad_len = max_len - len(input_sequence)

    tokens = tokenizer.convert_tokens_to_ids(input_sequence)
    tokens += [0] * pad_len
    pad_masks = [1] * len(input_sequence) + [0] * pad_len
    segment_ids = [0] * max_len
    
    all_tokens.append(tokens)
    all_masks.append(pad_masks)
    all_segments.append(segment_ids)
    

In [ ]:
!pip install -q tf-nightly

In [ ]:
_, sequence_output = bert_layer([tokens])
clf_test = sequence_output[:, 0, :]

In [ ]:
train_input = bert_encode(Xtrain, tokenizer, max_len=320)
test_input = bert_encode(Xtest, tokenizer, max_len=320)

In [ ]:
ytrain.shape

In [ ]:
test_input2 = test_model(Xtest)

In [ ]:
len(test_input)

## Creating Traing Model

In [ ]:
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
def build_model(bert_layer, max_len=2048):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    hidden1 = Dense(8, activation='relu')(clf_output)
    hidden2 = Dense(16, activation='relu')(hidden1)
    #hidden3 = Dense(64, activation='relu')(hidden2)
    out = Dense(1, activation='sigmoid')(hidden2)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=2e-6), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model 

In [ ]:
model = build_model(bert_layer, max_len=320)
model.summary()

In [ ]:
max_len = 512
input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

_, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
clf_output = sequence_output[:, 0, :]
clf_output

In [ ]:
Ytrain= Ytrain.fillna('Fake')
Ytest=Ytest.fillna("Fake")

In [ ]:
from sklearn.preprocessing import LabelEncoder
enocder = LabelEncoder()
ytrain =  enocder.fit_transform(Ytrain)
ytest =  enocder.fit_transform(Ytest)

In [ ]:
from keras.utils import to_categorical
ytrain = to_categorical(ytrain,6)
ytest = to_categorical(ytest,6)

In [ ]:
ytrain[0]

In [ ]:
Ytrain.unique()

In [ ]:
train_history = model.fit(
    train_input, Ytrain,
    validation_split=0.1,
    epochs=3,
    batch_size=16
)
model.evaluate(test_input,Ytest)

re = model.predict(test_input)
s = np.round(re,0)

from sklearn.metrics import precision_recall_curve,auc
#prob_pr = model_c.predict_proba(test)
precision,recall,thrs = precision_recall_curve(Ytest,re)
print("AUC PR Score :",auc(recall,precision))

from sklearn.metrics import precision_score, f1_score, recall_score
print("Precision :",precision_score(Ytest,s))
print("Recall Score :",recall_score(Ytest,s))
print("F1 Score :",f1_score(Ytest,s))

from sklearn.metrics import auc, roc_auc_score
print("AUC ROC Score :",roc_auc_score(Ytest,re))

In [ ]:
Ytrain.unique()

In [ ]:
re = model.predict(test_input)

In [ ]:
fl = re.flatten()

In [ ]:
s = np.round(re,0)

In [ ]:
from sklearn.metrics import precision_recall_curve,auc
#prob_pr = model_c.predict_proba(test)
precision,recall,thrs = precision_recall_curve(Ytest,re)
print("AUC PR Score :",auc(recall,precision))

from sklearn.metrics import precision_score, f1_score, recall_score
print("Precision :",precision_score(Ytest,s))
print("Recall Score :",recall_score(Ytest,s))
print("F1 Score :",f1_score(Ytest,s))

from sklearn.metrics import auc, roc_auc_score
print("AUC ROC Score :",roc_auc_score(Ytest,re))

In [ ]:
from sklearn.metrics import auc, roc_auc_score
roc_auc_score(Ytest,re)

In [ ]:
s

## References
https://towardsdatascience.com/bert-in-keras-with-tensorflow-hub-76bcbc9417b
https://medium.com/analytics-vidhya/bert-in-keras-tensorflow-2-0-using-tfhub-huggingface-81c08c5f81d8
https://github.com/strongio/keras-bert/blob/master/keras-bert.ipynb
https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/
http://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/
https://medium.com/@vineet.mundhra/loading-bert-with-tensorflow-hub-7f5a1c722565
https://towardsdatascience.com/building-a-search-engine-with-bert-and-tensorflow-c6fdc0186c8a
https://github.com/google-research/bert/blob/master/predicting_movie_reviews_with_bert_on_tf_hub.ipynb
https://gist.github.com/vineetm

## Bert Google Research
https://github.com/google-research/bert

## Bert Models
https://tfhub.dev/s?q=bert

## Note
Feel free to ask questions, initiate discussion on this topic. Please upvote the discussion and notebook to help me make more such contributions.